In [13]:
import numpy as np
import pandas as pd
import time
import random
from helper_functions import *

In [ ]:
#problem_instance = "inst_tuning\heur040_n_300_m_13358"
instance_type = "test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur002_n_100_m_3274" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"

In [ ]:
nodes, edges, s = create_problem_instance(path)

In [ ]:
is_splex(nodes = nodes, plex_number = 2, s=s)

In [41]:
def construction_heuristic(nodes_orig, edges_orig, s):
    nodes = nodes_orig.copy()
    edges = edges_orig.copy()
    
    start = time.time()
    
    existing_edges = edges.loc[edges["w"]<0].sort_values("w")

    for index, row in existing_edges.iterrows():
        # check if this edge was already added, then we can continue to next one
        if edges.loc[(edges["n1"]==row["n1"]) & (edges["n2"]==row["n2"]), "e"].values[0] == 1:
            continue
        # get plex assignment of both nodes
        n1_plex = nodes.loc[nodes["node_number"]==row["n1"], "splex"].values[0]
        n2_plex = nodes.loc[nodes["node_number"]==row["n2"], "splex"].values[0]
        # all nodes that would be in the plex if we merged it
        nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
        # all edges we would want anyway as they were in original assignment
        edges_to_add = edges.loc[(edges["n1"].isin(nodes_in_plex)) & 
                                        (edges["n2"].isin(nodes_in_plex)) & # only edges within the potential splex
                                       (edges["w"]<=0) & # that we want to add anyway
                                       (edges["e"]==0)] #that have not been added yet

        number_of_nodes = len(nodes_in_plex)
        edges_missing = False
        needed_edges_by_node: dict = dict()

        # check if it would be a valid splex if we merge by checking node degrees
        for node in nodes_in_plex:
            node_degree = nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]
            if node_degree < (number_of_nodes - s):
                # if we can reach the node degree by only using the edges we want to add anyway, then everything is fine
                num_potential_edges = edges_to_add.loc[(edges_to_add["n1"]==node)|
                                                             (edges_to_add["n2"]==node)].shape[0]
                new_degree = node_degree + num_potential_edges
                needed_edges = number_of_nodes - s - new_degree
                if needed_edges > 0:
                    edges_missing = True
                    needed_edges_by_node[node] = needed_edges
                    break

        # TODO: Also add 'bad' edges if it decreases total weight
        if False: #edges_missing:
            totalWeight = edges_to_add["w"].sum()
            
            nodes_to_process = needed_edges_by_node.keys()
            
            candidates = edges.loc[(edges["n1"].isin(nodes_in_plex)) & 
                                    (edges["n2"].isin(nodes_in_plex)) & # only edges within the potential splex
                                    (edges["w"]>0) & # that we do not add automatically
                                    (edges["e"]==0) & # that have not been added yet
                                    (edges["n1"].isin(nodes_to_process) |
                                    edges["n2"].isin(nodes_to_process))] # that have at least one node with missing edges
            candidates = candidates.sort_values(by=['w'])

            #print("edges" + str(edges_to_add))
            #print("candidates" + str(candidates))

            bad_edges_to_add = pd.DataFrame(columns=["n1","n2","e","w"])

            while not candidates.empty and (totalWeight < 0):
                current = candidates.iloc[0]
                candidates = candidates.iloc[1:]

                #print(len(candidates.index))
                if (not (current["n1"] in needed_edges_by_node)) \
                   and (not (current["n2"] in needed_edges_by_node)):
                   continue 
                
                #print("add edge " + str(current))
                #print(type(current))
                
                bad_edges_to_add = pd.concat([bad_edges_to_add, current.to_frame().T])
                #print("bad " + str(bad_edges_to_add))
                edge_weight = current["w"]

                node1 = current["n1"]
                if node1 in needed_edges_by_node:
                    needed_edges_by_node[node1] -= 1
                    if needed_edges_by_node[node1] == 0:
                        needed_edges_by_node.pop(node1)
                        
                node2 = current["n2"]
                if node2 in needed_edges_by_node:
                    needed_edges_by_node[node2] -= 1
                    if needed_edges_by_node[node2] == 0:
                        needed_edges_by_node.pop(node2)
                        
                totalWeight += edge_weight
                #print ("totalWeight: " + str(totalWeight))
                #print ("candidates: " + str(len(candidates.index)))

            if candidates.empty and totalWeight < 0:
                edges_missing = False
                edges_to_add = pd.concat([edges_to_add, bad_edges_to_add])

                for node in nodes_in_plex:
                    node_degree = nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]
                    if node_degree < (number_of_nodes - s):
                        # if we can reach the node degree by only using the edges we want to add anyway, then everything is fine
                        num_potential_edges = edges_to_add.loc[(edges_to_add["n1"]==node)|
                                                               (edges_to_add["n2"]==node)].shape[0]
                        new_degree = node_degree + num_potential_edges
                        needed_edges = number_of_nodes - s - new_degree
                        if needed_edges > 0:
                            edges_missing = True
                            needed_edges_by_node[node] = needed_edges
                            break                

        # if it would be a valid splex, actually merge it
        if not edges_missing:
            # merge them
            nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
            # include all edges we want to add
            for index, oedge in edges_to_add.iterrows():
                edges.loc[(edges["n1"]==oedge["n1"]) & (edges["n2"]==oedge["n2"]), "e"] = 1
                # update node info (degree and node impact)
                nodes.loc[nodes["node_number"]==oedge["n1"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n2"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n1"], "node_impact"] -= abs(oedge["w"])
                nodes.loc[nodes["node_number"]==oedge["n2"], "node_impact"] -= abs(oedge["w"])
    print(round(time.time()-start, 2), "seconds")
    score = edges.loc[((edges["e"]==0)&(edges["w"]<=0))| 
                        ((edges["e"]==1)&(edges["w"]>0)), ["w"]].abs().sum()
    return(nodes, edges, score)

In [20]:
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes,edges,s)
nodes

0.11 seconds


,node_number,node_impact,current_degree,splex
0,1,21.0,0,1
1,2,21.0,0,2
2,3,19.0,0,3
3,4,22.0,0,4
4,5,13.0,0,5
5,6,12.0,0,6
6,7,17.0,0,7
7,8,11.0,0,8
8,9,22.0,0,9
9,10,22.0,0,10


In [21]:
print(len(constr_nodes["splex"].unique()))
constr_nodes

3


,node_number,node_impact,current_degree,splex
0,1,11.0,4,1
1,2,8.0,4,1
2,3,8.0,4,1
3,4,11.0,2,4
4,5,3.0,4,1
5,6,2.0,4,1
6,7,9.0,2,4
7,8,7.0,1,8
8,9,18.0,1,8
9,10,13.0,2,4


In [22]:
def randomized_greedy(nodes_orig, edges_orig, s, alpha=0.5, random_seed = None):
    nodes = nodes_orig.copy()
    edges = edges_orig.copy()
    
    if random_seed is not None:
        random.seed(random_seed)
    start = time.time()
    
    existing_edges = edges.loc[edges["w"]<0].sort_values("w") # this is our candidate list
    
    while existing_edges.shape[0]>0:
        #build restricted candidate list
        costs_threshold = max(existing_edges["w"]) + alpha * (min(existing_edges["w"]) - max(existing_edges["w"]))
        rcl = existing_edges.loc[existing_edges["w"]>= costs_threshold]
        # pick one edge at random
        row = rcl.sample(1)
            
        # get plex assignment of both nodes
        n1_plex = nodes.loc[nodes["node_number"]==row["n1"].values[0], "splex"].values[0]
        n2_plex = nodes.loc[nodes["node_number"]==row["n2"].values[0], "splex"].values[0]
        # all nodes that would be in the plex if we merged it
        nodes_in_plex = nodes.loc[(nodes["splex"]==n1_plex) | (nodes["splex"] ==n2_plex), "node_number"].values
        # all edges we would want anyway as they were in original assignment
        edges_to_add = edges.loc[(edges["n1"].isin(nodes_in_plex)) & 
                                        (edges["n2"].isin(nodes_in_plex)) & # only edges within the potential splex
                                       (edges["w"]<=0) & # that we want to add anyway
                                       (edges["e"]==0)] #that have not been added yet

        number_of_nodes = len(nodes_in_plex)
        edges_missing = False
        needed_edges_by_node = dict()

        # check if it would be a valid splex if we merge by checking node degrees
        for node in nodes_in_plex:
            node_degree = nodes.loc[nodes["node_number"]==node,"current_degree"].values[0]
            if node_degree < (number_of_nodes - s):
                # if we can reach the node degree by only using the edges we want to add anyway, then everything is fine
                num_potential_edges = edges_to_add.loc[(edges_to_add["n1"]==node)|
                                                             (edges_to_add["n2"]==node)].shape[0]
                
                needed_edges = number_of_nodes - s - node_degree - num_potential_edges
                if needed_edges > 0:
                    edges_missing = True
                    needed_edges_by_node[node] = needed_edges
                    break

        # TODO: Also add 'bad' edges if it decreases total weight
        if edges_missing:
            totalWeight = edges_to_add["w"]


        # if it would be a valid splex, actually merge it
        if not edges_missing:
            # merge them
            nodes.loc[nodes["splex"]==n2_plex, "splex"] = n1_plex
            # include all edges we want to add
            for index, oedge in edges_to_add.iterrows():
                edges.loc[(edges["n1"]==oedge["n1"]) & (edges["n2"]==oedge["n2"]), "e"] = 1
                # update node info (degree and node impact)
                nodes.loc[nodes["node_number"]==oedge["n1"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n2"], "current_degree"] +=1
                nodes.loc[nodes["node_number"]==oedge["n1"], "node_impact"] -= abs(oedge["w"])
                nodes.loc[nodes["node_number"]==oedge["n2"], "node_impact"] -= abs(oedge["w"])

        # remove the edge from the candidate list
        existing_edges = existing_edges.drop(existing_edges[(existing_edges.n1 == row.n1.values[0])&
                                                            (existing_edges.n2 == row.n2.values[0])].index)
    score = edges.loc[((edges["e"]==0)&(edges["w"]<=0))| 
                        ((edges["e"]==1)&(edges["w"]>0)), ["w"]].abs().sum()    
    print(round(time.time()-start, 2), "seconds")
    return(nodes, edges, score)

In [23]:
rand_nodes, rand_edgesm, rand_score = randomized_greedy(nodes, edges, s, alpha = 0.75)

0.15 seconds


In [24]:
rand_nodes

,node_number,node_impact,current_degree,splex
0,1,12.0,4,1
1,2,5.0,4,1
2,3,13.0,2,3
3,4,13.0,4,1
4,5,3.0,4,1
5,6,6.0,4,1
6,7,16.0,1,7
7,8,10.0,1,7
8,9,13.0,2,3
9,10,15.0,2,3


In [27]:
print(len(rand_nodes["splex"].unique()))
rand_nodes

3


,node_number,node_impact,current_degree,splex
0,1,12.0,4,1
1,2,5.0,4,1
2,3,13.0,2,3
3,4,13.0,4,1
4,5,3.0,4,1
5,6,6.0,4,1
6,7,16.0,1,7
7,8,10.0,1,7
8,9,13.0,2,3
9,10,15.0,2,3


In [42]:
instance_type = "inst_competition/"#"test_instances/" #"inst_competition/" #"inst_tuning/" # 
problem_instance = "heur051_n_300_m_20122" #"heur040_n_300_m_13358" #"heur001_n_10_m_31" 
path = "data/"+instance_type + problem_instance +".txt"
nodes, edges, s = create_problem_instance(path)
print(edges)
constr_nodes, constr_edges, constr_score = construction_heuristic(nodes, edges, s)
write_solution(constr_edges, problem_instance, algorithm = "construction")
print(constr_edges)

        n1   n2  e  w
0        1    2  0 -1
1        1    3  0 -7
2        1    4  0 -2
3        1    5  0 -2
4        1    6  0 -3
...    ...  ... .. ..
44845  297  299  0 -1
44846  297  300  0 -1
44847  298  299  0 -2
44848  298  300  0 -1
44849  299  300  0 -5

[44850 rows x 4 columns]
116.88 seconds


TypeError: write_solution() got an unexpected keyword argument 'algorithm'

In [43]:
constr_score

w    46277
dtype: int64